In [1]:
# imports
import numpy as np ; import re ; import pandas as pd

# load files
PWD = "/Users/antongolles/Documents/Work/ITU-Research/DBS_textual/"
relative_location = "../data/Shareholder_decks/"
date_latest = '2022-05-18'

file = np.load(relative_location+f'text_from_shareholder_decks_{date_latest}.npz', allow_pickle=True)
text_dict = file[file.files[0]].item()

file = np.load('../data/keywords_dict.npz', allow_pickle=1)
keyword_dict = file[file.files[0]].item()

prevalence_df = pd.read_csv('../data/matches_on_words_df{}.csv'.format('2022-06-13'), index_col=0)

In [2]:
def get_locations_of_keywords(text_file=text_dict['google']['google_2020_Q2']['text'],
                             category ='DBS'):
    locations_of_keywords = {'words':[],'starts' : [], 'stops': []}
    for index in keyword_dict[category]:
        #for form in ['text','lemmas','stems']:
        p = re.compile(keyword_dict[category][index]['text'])
        for m in p.finditer(text_file):
            locations_of_keywords["words"].append(m.group())
            locations_of_keywords["starts"].append(m.start())
            locations_of_keywords["stops"].append(m.end())
    return locations_of_keywords

def prevalence(keyword):
    return list(prevalence_df[prevalence_df['index'] == keyword]['count'])[0]/prevalence_df["count"].max()
    
def get_distances_and_prevalence(text_file=text_dict['google']['google_2020_Q2']['text']):
    '''a list of distances from the start and end of each scope keyword to the nearest dbs keyword'''
    distances_and_prevalence = {}
    dbs = get_locations_of_keywords(category=list(keyword_dict.keys())[0])

    for category in keyword_dict.keys():
        if category == 'DBS':
            pass
        else:
            distances_and_prevalence[category] = {}
            locs = get_locations_of_keywords(text_file, category=category)
            distance_to_dbs_keywords = [[min(abs(np.array(dbs['starts']+dbs['stops'])-i)), min(abs(np.array(dbs['starts']+dbs['stops'])-j)) ] for i, j in zip(locs['starts'], locs['stops'])]
            distances_and_prevalence[category]["distances"] = np.min(np.array(distance_to_dbs_keywords), axis=1)
            distances_and_prevalence[category]["prevalence"] = [prevalence(word) for word in locs['words']]
    return distances_and_prevalence



In [7]:
locs = get_locations_of_keywords(text_file, category=category)
locs

NameError: name 'text_file' is not defined

In [11]:
get_distances_and_prevalence()

{'Scope of Digital Business Strategy': {'distances': array([ 97,  72,   1, 642, 625, 694, 580, 382, 298,  77, 208, 231, 212,
         417, 523, 176,   1, 642, 625, 694, 580, 382, 298,  77, 208, 231,
         212, 417, 523, 139, 330, 575]),
  'prevalence': [0.23321917808219178,
   0.23321917808219178,
   0.9863013698630136,
   0.9863013698630136,
   0.9863013698630136,
   0.9863013698630136,
   0.9863013698630136,
   0.9863013698630136,
   0.9863013698630136,
   0.9863013698630136,
   0.9863013698630136,
   0.9863013698630136,
   0.9863013698630136,
   0.9863013698630136,
   0.9863013698630136,
   0.22910958904109588,
   0.9863013698630136,
   0.9863013698630136,
   0.9863013698630136,
   0.9863013698630136,
   0.9863013698630136,
   0.9863013698630136,
   0.9863013698630136,
   0.9863013698630136,
   0.9863013698630136,
   0.9863013698630136,
   0.9863013698630136,
   0.9863013698630136,
   0.9863013698630136,
   0.5061643835616438,
   0.14726027397260275,
   0.43767123287671234]},
 'S

# big question: *How do we convert this to a score?*
**positiv correlation with amount of keywords**
* sum?

**negativ correlation with distances**
* exponential decay

**negativ correlation with keyword prevelance in entire set**
* multiply by percentage of reports not containing keyword, ie 1 minus prevelance: $1-P({k})$

$$
S_\text{category} = \sum_\text{$k$ in keywordMatches in category} \left( e^{-a(d-1)}\cdot(1-P(k))\right)
$$

**key:**
* $S$ : Score
* $k$ : keyword
* $P$ : keyword prevalence
* $a$ : factor on decay speed
* $d$ : distance to DBS keyword




In [16]:
def getScore(distances_and_prevalence, a=.5):
    score = {}
    for category in distances_and_prevalence.keys():
        arr = np.array(list(distances_and_prevalence[category].values()))
        arr[0] = np.exp(-a*(arr[0]-1))
        arr[1] = 1-arr[1]
        score[category] = sum(arr[0]*arr[1])
        
        
    return score

# Now lets get the score for all the shareholder decks

In [37]:
data = {}
for report in text_dict['google'].keys():
    data_ = {}
    report_ = text_dict['google'][report]['text']
    data_['length'] = len(report_)
    
    dist_prev = get_distances_and_prevalence(report_)
    
    for cat in dist_prev:
        data_[f'avg_dist_{cat.split()[0]}'] = np.mean(dist_prev[cat]['distances'])
    
    
    
    data[report] = {**getScore(dist_prev), **data_} 
    
df = pd.DataFrame.from_dict(data).sort_index().T

In [39]:
df.reset_index(inplace = True)

def qname_to_float(x):
    a = x.split('_')
    year = float(a[1])
    Q =float(x[-1])-1
    return year+0.25*Q

df['time'] = df['index'].apply(qname_to_float)
df.set_index('time', inplace=True)
df.drop(columns=['index'], inplace=True)

df


,Scale of Digital Business Strategy,Scope of Digital Business Strategy,Sources of Value Creation and Capture,Speed of Digital Business Strategy,avg_dist_Scale,avg_dist_Scope,avg_dist_Sources,avg_dist_Speed,length
time,,,,,,,,,
2020.75,8.318106e-01,3.688312e-04,3.515121,1.419853,1028.625000,515.805556,493.178571,475.035088,25774.0
2018.00,3.007812e-01,5.589577e-02,4.318382,2.364724,445.547619,420.352941,344.093960,314.507937,23893.0
2018.50,7.318748e-01,3.804916e-13,2.827412,0.600827,294.481481,440.000000,274.015625,377.639344,20249.0
2018.25,1.932573e-01,7.477860e-03,2.921603,0.714967,419.382353,373.954545,386.985185,332.676923,23634.0
2020.25,2.492775e-10,2.739726e-02,1.292108,0.309168,301.714286,349.031250,248.435115,384.000000,22251.0
2020.50,2.744945e-02,1.407295e+00,2.668973,0.273985,268.107143,425.718750,280.937984,343.000000,22174.0
2020.00,3.021525e-08,8.519432e-03,2.669577,0.852217,312.285714,418.531250,262.272059,353.826923,21676.0
2018.75,1.384147e+00,3.627112e-05,4.072945,1.789497,303.642857,476.073171,263.821705,379.507937,21823.0
2019.25,3.885518e-01,4.352587e-09,1.566711,2.281242,502.685714,376.809524,334.583942,311.117647,23948.0


In [41]:
import seaborn as sns

sns.pairplot(df)

In [15]:
data = {}
i = 0
for company in text_dict.keys():
    if i<1:
        data[company] = {}
        for report in text_dict[company].keys():
            try:
                report_formatted = ''.join(report.split('_')[1:])
                data[company][report_formatted] = getScore(get_distances_and_prevalence(text_dict[company][report]['text']))
            except:
                print(report)

    i += 1       
pd.DataFrame.from_dict(data).sort_index().T

apple_2017_Q2


,2017Q1,2017Q3,2017Q4,2018Q1,2018Q2,2018Q3,2018Q4,2019Q1,2019Q2,2019Q3,...,2020Q1,2020Q2,2020Q3,2020Q4,2021Q1,2021Q2,2021Q3,2021Q4,2022Q1,2022Q2
apple,0.045815,1.762172,0.82719,1.011644,0.362945,0.558414,1.625282,1.299915,1.66952,1.281028,...,2.094207,0.584305,1.055516,1.028349,4.824927,0.78656,0.44079,0.352916,0.202791,0.976185


# extra columns:
* avg distance
* number of keywords found
* score for each category
* length of text (characters)

,apple,paccar,starbucks,moderna,honeywell
2020Q4,1.028349,5.257499,5.157410,3.061365,1.038047
2022Q1,0.202791,1.179227,8.748137,NaN,0.602907
2020Q3,1.055516,NaN,4.313471,2.038580,2.024903
2020Q2,0.584305,NaN,4.313471,0.841160,2.919194
2020Q1,2.094207,NaN,4.169138,4.196209,0.994633
2019Q3,1.281028,NaN,4.280781,0.668218,0.126574
2019Q2,1.669520,NaN,4.508376,0.425605,0.306680
2021Q4,0.352916,1.277661,2.837307,3.658519,2.146858
2019Q1,1.299915,NaN,1.945632,2.051170,1.750490
2021Q1,4.824927,2.967633,6.773552,2.756257,0.994633
